# SAM 3D Body (3DB) Mesh Alignment to SAM 3D Object Scale

This notebook processes a single 3DB mesh and aligns it to the SAM 3D Objects scale.

**Input Data:**
- `images/human_object/image.jpg` - Input image for MoGe
- `meshes/human_object/3DB_results/mask_human.png` - Human mask
- `meshes/human_object/3DB_results/human.ply` - Single 3DB mesh in OpenGL coordinates
- `meshes/human_object/3DB_results/focal_length.json` - 3DB focal length

**Output:**
- `meshes/human_object/aligned_meshes/human_aligned.ply` - Aligned 3DB mesh in OpenGL coordinates

In [ ]:
import os
import torch
import matplotlib.pyplot as plt
from PIL import Image
from mesh_alignment import process_and_save_alignment

device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(f"Using device: {device}")
PATH = os.getcwd()
print(f"Current working directory: {PATH}")

# Please inference the SAM 3D Body (3DB) Repo (https://github.com/facebookresearch/sam-3d-body) to get the 3DB Results
image_path = f"{PATH}/images/human_object/image.png"
mask_path = f"{PATH}/meshes/human_object/3DB_results/mask_human.png"
mesh_path = f"{PATH}/meshes/human_object/3DB_results/human.ply"
focal_length_json_path = f"{PATH}/meshes/human_object/3DB_results/focal_length.json"
output_dir = f"{PATH}/meshes/human_object/aligned_meshes"
os.makedirs(output_dir, exist_ok=True)


## 0. Inference and Save SAM 3D Objects

In [ ]:
# Please inference SAM 3D Objects Repo with https://github.com/facebookresearch/sam-3d-objects/blob/main/notebook/demo_multi_object.ipynb
# The above notebook will apply the generated layout to the generated objects, and same them as ply. 
# Then, this cell will load the posed SAM 3D Objects and transform them into the OpenGL coordinate system, which is the same system as SAM 3D Body. 
import numpy as np
import open3d as o3d

# Load PLY file
input_path = 'gaussians/human_object_posed.ply'
output_path = 'meshes/human_object/3Dfy_results/0.ply'
mesh = o3d.io.read_point_cloud(input_path)
points = np.asarray(mesh.points)

# Transform to OpenGL coordinate system. 
points[:, [0, 2]] *= -1  # flip x and z
mesh.points = o3d.utility.Vector3dVector(points)
o3d.io.write_point_cloud(output_path, mesh)

## 1. Load and Display Input Data

In [ ]:
input_image = Image.open(image_path)
mask = Image.open(mask_path).convert('L')
fig, axes = plt.subplots(1, 2, figsize=(10, 5))
axes[0].imshow(input_image)
axes[0].set_title('Input Image')
axes[0].axis('off')
axes[1].imshow(mask, cmap='gray')
axes[1].set_title('Mask')
axes[1].axis('off')
plt.tight_layout()
plt.show()

## 2. Process and Save Aligned Mesh

In [ ]:

success, output_mesh_path, result = process_and_save_alignment(
    mesh_path=mesh_path,
    mask_path=mask_path,
    image_path=image_path,
    output_dir=output_dir,
    device=device,
    focal_length_json_path=focal_length_json_path
)

if success:
    print(f"Alignment completed successfully! Output: {output_mesh_path}")
else:
    print("Alignment failed!")

## 3. Interactive 3D Visualization


In [ ]:
from mesh_alignment import visualize_meshes_interactive

aligned_mesh_path = f"{PATH}/meshes/human_object/aligned_meshes/human_aligned.ply"
dfy_mesh_path = f"{PATH}/meshes/human_object/3Dfy_results/0.ply"

demo, combined_glb_path = visualize_meshes_interactive(
    aligned_mesh_path=aligned_mesh_path,
    dfy_mesh_path=dfy_mesh_path,
    share=True
)